In [2]:
import numpy as np
import pandas as pd
import nibabel as nib

Load a pre-made 4D image containing binary lesion masks from multiple patients.

In [3]:
mask_stack_path = '/home/despoB/lesion/anat_preproc/lesion_mask_mni_stack.nii.gz'

In [4]:
mask_stack_img = nib.load(mask_stack_path)

In [5]:
mask_stack_data = mask_stack_img.get_data()

In [6]:
mask_stack_data.shape

(91, 109, 91, 64)

Get the inverse affine transform for the image, so we can go from image-space coordinates to voxel-data coordiantes.

In [7]:
mask_stack_img.affine

array([[  -2.,   -0.,    0.,   90.],
       [  -0.,    2.,   -0., -126.],
       [   0.,    0.,    2.,  -72.],
       [   0.,    0.,    0.,    1.]])

In [8]:
inverse_affine = np.linalg.inv(mask_stack_img.affine)

Confirm that our coordinate translation works by checking against a known relationship (from FSLView).

MNI coordinate (28, 36, 2) = Voxel coordinate (31, 81, 37)

In [9]:
nib.affines.apply_affine(inverse_affine, [28, 36, 2])

array([ 31.,  81.,  37.])

To be double sure, test our transformed coordinates on real data. 

The patient 191 has a lesioned voxel at MNI coordinate (-36, 2, -2), but moving more than 2mm (1 voxel) in any direction puts you in empty space.

In [12]:
mask_path_191 = '/home/despoB/lesion/anat_preproc/191/191_mask_mni.nii.gz'

In [13]:
mask_img_191 = nib.load(mask_path_191)

In [14]:
mask_data_191 = mask_img_191.get_data()

There should be damage here..

In [15]:
nib.affines.apply_affine(inverse_affine, [-36, 2, -2])

array([ 63.,  64.,  35.])

In [16]:
mask_data_191[63, 64, 35]

1.0

But not in any of these places...

In [17]:
nib.affines.apply_affine(inverse_affine, [-40, 2, -2])

array([ 65.,  64.,  35.])

In [18]:
mask_data_191[65, 64, 35]

0.0

In [19]:
nib.affines.apply_affine(inverse_affine, [-36, -2, -2])

array([ 63.,  62.,  35.])

In [20]:
mask_data_191[63, 62, 35]

0.0

In [21]:
nib.affines.apply_affine(inverse_affine, [-36, 2, 2])

array([ 63.,  64.,  37.])

In [22]:
mask_data_191[63, 64, 37]

0.0

Looks like our coordinate conversion is working. These were lazy checks, but lazy is better than nothing.

Next, we need to map 3D voxel-data coordinates to 1D (flattened) coordinates.

First, create a small 3D array to test things on.

In [23]:
test_data = np.random.rand(3, 3, 3)

In [24]:
test_data.shape

(3, 3, 3)

In [25]:
test_data

array([[[ 0.28608054,  0.96612575,  0.0491338 ],
        [ 0.4174178 ,  0.78697586,  0.40856223],
        [ 0.67486548,  0.0242388 ,  0.83784137]],

       [[ 0.86366276,  0.33605138,  0.10813308],
        [ 0.20377529,  0.59884482,  0.95976346],
        [ 0.22732127,  0.62403217,  0.06885925]],

       [[ 0.03200667,  0.33162971,  0.49775652],
        [ 0.81039875,  0.8548744 ,  0.86193231],
        [ 0.71003148,  0.22703014,  0.6580786 ]]])

Pull an arbitrary item using 3D indexing.

In [26]:
test_data[0,2,1]

0.024238799862456029

The `ravel_multi_index` function translates a multi-dimensional index into the equivalent 1D index of a raveled array.

In [27]:
np.ravel_multi_index([0,2,1], (3,3,3))

7

In [28]:
test_data.ravel()[7]

0.024238799862456029

Now let's check this works on our image data.

In [29]:
mask_data_191[63, 64, 35]

1.0

In [30]:
np.ravel_multi_index([63, 64, 35], mask_data_191.shape)

630756

In [31]:
mask_data_191.ravel()[630756]

1.0

Now that we've got indexing working using NumPy arrays, let's translate things into Pandas so we can have labeled rows.

Construct a test DF with image vectors from 10 patients. We do it iteratively, adding one patient at a time. 

We'll also do some code profiling to see how much of a hit the system will take when doing these operations.

In [36]:
%load_ext memory_profiler

In [37]:
%%memit
for i in range(10):
    pdata = mask_stack_data[...,i].ravel()
    try:
        mask_data_df['10'+str(i)] = pdata
    except:
        mask_data_df = pd.DataFrame({'10'+str(i):pdata})

peak memory: 621.07 MiB, increment: 82.59 MiB


In [38]:
mask_data_df.head()

,100,101,102,103,104,105,106,107,108,109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Now that we have the DF, let's search it.

In [39]:
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

['102', '103', '104', '105', '106']

In [40]:
%%timeit
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

1000 loops, best of 3: 1.01 ms per loop


In [41]:
%memit
test_results = mask_data_df.loc[630756,:] == 1
list(test_results[test_results == True].index)

peak memory: 721.48 MiB, increment: 0.00 MiB


['102', '103', '104', '105', '106']

Sure enough, visual inspection confirms that those patients have lesions at that coordinate.

Let's see if it is much slower when we tweak things so the outputs a list of patients directly.

In [42]:
%%timeit
mask_data_df.T[mask_data_df.T[630756] == 1].index

10 loops, best of 3: 35 ms per loop


Okay, so it looks like we definitely don't want to just transpose the DF on the fly. What if we do that beforehand.

In [43]:
mask_data_t = mask_data_df.T

In [44]:
%%timeit
mask_data_t[mask_data_t[630756] == 1].index

100 loops, best of 3: 17 ms per loop


Hmm, okay. It seems like the slowdown comes from indexing over columns instead of rows. Back to our original plan.

We want our mask database to be persistent, but fast to read and write, so we're going to use PyTables to store it as HDF5.

First, create an HDFstore object and add our DF to it.

In [223]:
store = pd.HDFStore('mask_data.h5')

In [224]:
store.put('df', mask_data_df, format='t')

In [225]:
store

<class 'pandas.io.pytables.HDFStore'>
File path: mask_data.h5
/df            frame_table  (typ->appendable,nrows->902629,ncols->10,indexers->[index])

In [226]:
store.df.head()

,100,101,102,103,104,105,106,107,108,109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We can close the store to free up memory when we're not using the mask data.

In [227]:
store.close()
del store

When working with data stored as HD5, we can select only the rows/columns we want to be read into memory.

This makes searching really fast.

In [289]:
flat_index = 630756

In [292]:
%%timeit
restore = pd.HDFStore('mask_data.h5')
vox_row = restore.select('df', start=flat_index, stop=flat_index+1)
res = vox_row.T.iloc[:,0] == 1
patients = list(res[res == True].index)
restore.close()

100 loops, best of 3: 10.8 ms per loop


Much faster than reading the whole DF into memory.

In [293]:
%%timeit
store = pd.HDFStore('mask_data.h5')
test_results = store.df.loc[630756,:] == 1
patients = list(test_results[test_results == True].index)
store.close()

The slowest run took 8.58 times longer than the fastest. This could mean that an intermediate result is being cached.
1 loop, best of 3: 119 ms per loop


Now that we've got things working, we'll put everything into functions.

In [207]:
import os 
def add_patient_mask(patient_number, mask_path, store_path):
    """
    Add a new patient/mask to the damage database.
    
    Parameters
    ----------
    patient_number : str
    mask_path : str
        Path the patient's lesion mask. Must be in MNI space,
        with 2mm voxels, and have RPI orientation.
    datastore : str
        Path to a Pandas HDF5store (e.g. file.h5)
    """
    # Check inputs
    assert isinstance(patient_number, str)
    assert os.path.exists(mask_path)
    
    # Load the mask image.
    mask_data = nib.load(mask_path).get_data()
    
    # Open the HD5 store, add a column for the new patient, then close the store.
    store = pd.HDFStore(store_path)
    store.df[patient_number] = mask_data.ravel()
    print(store.df.columns)
    store.flush()
    store.close()

In [208]:
%%time
add_patient_mask('191', '/home/despoB/lesion/anat_preproc/191/191_mask_mni.nii.gz','mask_data.h5')

Index(['100', '101', '102', '103', '104', '105', '106', '107', '108', '109'], dtype='object')
CPU times: user 4 ms, sys: 1.04 s, total: 1.04 s
Wall time: 2.16 s


In [211]:
store = pd.HDFStore('mask_data.h5')

In [213]:
store.df['foo'] = 99

In [ ]:
store.

In [197]:
store.df.max()

100    1.0
101    1.0
102    1.0
103    1.0
104    1.0
105    1.0
106    1.0
107    1.0
108    1.0
109    1.0
dtype: float64

In [ ]:
store.flush()

In [ ]:
mask_data_df.to_hdf()

In [222]:
with pd.HDFStore('mask_data.h5') as store:
    store.df['foo'] = 99
    print(store.df.columns)

Index(['100', '101', '102', '103', '104', '105', '106', '107', '108', '109'], dtype='object')


In [217]:
restore['foo'] = 99

In [218]:
restore

,100,101,102,103,104,105,106,107,108,109,foo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,99


In [174]:
store.df

,100,101,102,103,104,105,106,107,108,109
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in range(10):
    pdata = mask_stack_data[...,i].ravel()
    try:
        mask_data_df['10'+str(i)] = pdata
    except:
        mask_data_df = pd.DataFrame({'10'+str(i):pdata})